# Create master df
Total observations = 50,000
<br>0 = 25,000, 15,000 from sentiment 140, 10,000 from amazon reviews
<br>1 = 25,000, 10,000 from sentiment 140, 5000 from consumer complaints, 10000 from amazon reviews
<br> All observations from sentiment 140_neutlabels

In [28]:
import pandas as pd
pd.set_option('display.max_colwidth', 0) #To display entire text content of a column
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'
from sklearn.model_selection import train_test_split
import os
from functools import reduce
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords 

In [4]:
os.chdir('D:\capstone_data\processed_csv\duplicates_rm')

In [12]:
#Read all dfs
sentiment_140_df = pd.read_csv('sentiment140_no_dplcts_textcol.csv')
sentiment_140_neutlabels =  pd.read_csv('sentiment_140_neutlabels_no_dplcts_textcol.csv')
consumer_complaints_df = pd.read_csv('consumer_complaints_no_dplcts_textcol.csv')
amazonreviews_df = pd.read_csv('amazonreviews_with_label_no_dplcts_textcol.csv')

In [5]:
#Extract 15,000 0s from sentiment 140
sentiment_140_0s_15k = sentiment_140_df[sentiment_140_df['label']==0].sample(
    n = 15000, replace = False, random_state = 7)
sentiment_140_0s_15k['label'].value_counts()

0    15000
Name: label, dtype: int64

In [6]:
#Extract 10,000 0s from amazon reviews
amazonreviews_0s_10k = amazonreviews_df[amazonreviews_df['label'] == 0].sample(
n = 10000, replace = False, random_state = 7)
amazonreviews_0s_10k['label'].value_counts()

0    10000
Name: label, dtype: int64

In [7]:
#Extract all 0s from sentiment_140_neutlabels
sentiment_140_neutlabels_0s = sentiment_140_neutlabels[sentiment_140_neutlabels['label'] == 0]
sentiment_140_neutlabels_0s['label'].value_counts()

0    321
Name: label, dtype: int64

In [8]:
#Concat all 0s in master df 
master_df = pd.concat([sentiment_140_0s_15k, amazonreviews_0s_10k, sentiment_140_neutlabels_0s],
                     axis = 0, ignore_index = True)

In [9]:
#Verify
display(master_df.tail(2))
display(master_df['label'].value_counts())

,text,label
25319,Ask Programming: LaTeX or InDesign?: submitted by calcio1 [link] [1 comment] http://tinyurl.com/myfmf7,0
25320,Ahhh... back in a *real* text editing environment. I &lt;3 LaTeX.,0


0    25321
Name: label, dtype: int64

In [7]:
#Create a function to extract data points from df
def extract_data_pts(df, label, n_pts):
    df = df[df['label'] == label].sample(
    n = n_pts, replace = False, random_state = 7)
    return df

In [11]:
#Extract 10,000 1s from sentiment 140
sentiment_140_1s_10k = extract_data_pts(sentiment_140_df,1,10000)
display(sentiment_140_1s_10k['label'].value_counts())
display(sentiment_140_1s_10k.head(2))

1    10000
Name: label, dtype: int64

,text,label
548091,MISSING MY BOYFRIEND,1
163918,@SquishT77 So sorry to hear about your mum.,1


In [12]:
#Extract 5,000 1s from consumer complaints
consumer_complaints_1s_5k = extract_data_pts(consumer_complaints_df,1,5000)
consumer_complaints_1s_5k['label'].value_counts()

1    5000
Name: label, dtype: int64

In [13]:
#Extract 10,000 1s from amazon
amazon_reviews_1s_10k = extract_data_pts(amazonreviews_df,1,10000)
amazon_reviews_1s_10k['label'].value_counts()

1    10000
Name: label, dtype: int64

In [14]:
#Extract all 1s from amazon_reviews_1s_10k
sentiment_140_neutlabels_1s = sentiment_140_neutlabels[sentiment_140_neutlabels['label'] == 1]
sentiment_140_neutlabels_1s['label'].value_counts()

1    177
Name: label, dtype: int64

In [15]:
#Concat all 4 dfs above and append to master_df
master_df = master_df.append(
    pd.concat([sentiment_140_1s_10k,consumer_complaints_1s_5k,
    amazon_reviews_1s_10k, sentiment_140_neutlabels_1s],
                          axis = 0, ignore_index = True)
    , ignore_index = True)

In [16]:
#Verify master_df
display(master_df.shape)
display(master_df.tail(2))
display(master_df['label'].value_counts())

(50498, 2)

,text,label
50496,"Trouble in Iran, I see. Hmm. Iran. Iran so far away. #flockofseagullsweregeopoliticallycorrect",1
50497,Reading the tweets coming out of Iran... The whole thing is terrifying and incredibly sad...,1


0    25321
1    25177
Name: label, dtype: int64

In [18]:
#Write master_df to csv
master_df.to_csv('master_df_snmnt_&cmplnts.csv', index = False)

# Data Cleaning 

In [6]:
#Read 
master_df = pd.read_csv('master_df_snmnt_&cmplnts.csv')
master_df.head()

,text,label
0,@vallywally LOL.. Damm U make commision when SHE wins.. NICEE.. BTW.. What do u play when ure there?,0
1,@k80bug Ohhh I used to make those when I was operations there,0
2,That is...if I can make it thru the west and southside of Montgomery without being stopped by the punkass cops.,0
3,"Good stuff med @ericries Strax ï¿½ver en timme lï¿½ng, check it! http://bit.ly/15Vumv Tack @emileifrem !!",0
4,Tomorrow I begin a two day shoot for an online retailer. I'll be posing in various garments... I love it!,0


## Data cleaning steps
1. Remove user handles - e.g. @broskiii OH SNAP YOU WORK......
2. Remove hashtags  - e.g. #at&amp;t is complete fail
3. Remove urls
4. Replace punctuations with whitespace - For intial model we will focus on key words to determine negative tweets.
5. Remove numbers
6. Trim repeated letters in a word - People tend to use mutiple letters to emphasize especially on twitter e.g. I am soooooooooo hungryyyyyyyyyyyyyyyyyy. Any letter repeating more than twice will be trimmed. The e.g. text will be modified to I am soo hungryy.
<br>
<br>It must be noted that it is very important to complete steps 1 through 3 before replacing punctuations with whitespace because they all include some form of punctuation like @,#,// etc. that will be removed and alphanumeric values are left unchanged. e.g. "http://twitpic.com/5ej4y - My favorite neurotic sister" will be transformed to "http   twitpic com 5ej4y   My favorite neurotic sister"


In [7]:
#Smaller sample from master to test our functions
df = master_df.head(25)
df

,text,label
0,@vallywally LOL.. Damm U make commision when SHE wins.. NICEE.. BTW.. What do u play when ure there?,0
1,@k80bug Ohhh I used to make those when I was operations there,0
2,That is...if I can make it thru the west and southside of Montgomery without being stopped by the punkass cops.,0
3,"Good stuff med @ericries Strax ï¿½ver en timme lï¿½ng, check it! http://bit.ly/15Vumv Tack @emileifrem !!",0
4,Tomorrow I begin a two day shoot for an online retailer. I'll be posing in various garments... I love it!,0
5,@hillkath Good morning.,0
6,"In Melbourne Australia, Thursday 7th May there is a Body Balance info night, tasting and free bottle! Direct message me email for details",0
7,@therealswizzz http://twitpic.com/5eyah - XOXO,0
8,@SashaKane have a happy day,0
9,@VivaLaCaro That was coolio! &lt;3 I left reviews and fatty fuel.,0


In [8]:
# Step 1. Remove user handles
df['clean_text'] = df['text'].apply(lambda x: re.sub("@[\w]*","", x))
#@ = literal match, @[] = @ followed by, \w = Aplhanumeric , * = 0 or more occurences
#Verify
df.head(2)

C:\Users\Anuroop\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,text,label,clean_text
0,@vallywally LOL.. Damm U make commision when SHE wins.. NICEE.. BTW.. What do u play when ure there?,0,LOL.. Damm U make commision when SHE wins.. NICEE.. BTW.. What do u play when ure there?
1,@k80bug Ohhh I used to make those when I was operations there,0,Ohhh I used to make those when I was operations there


In [9]:
# Step 2. Remove hashtags
df['clean_text'] = df['clean_text'].apply(lambda x: re.sub("#[\w]*","", x))
#Verify
df.tail(2)

C:\Users\Anuroop\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,text,label,clean_text
23,@mileycyrus hi miley i loved you're movie it was amazing !,0,hi miley i loved you're movie it was amazing !
24,still keep crocheting n thinking if i should watch #fringe as well #crochet,0,still keep crocheting n thinking if i should watch as well


In [10]:
#Step 3. Remove urls
df['clean_text'] = df['clean_text'].apply(lambda x: re.sub("http\S+", "", x))
# http = literal match, \S -all non white spaces, + = one or more occurences
#Verify
df.head(10)

C:\Users\Anuroop\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,text,label,clean_text
0,@vallywally LOL.. Damm U make commision when SHE wins.. NICEE.. BTW.. What do u play when ure there?,0,LOL.. Damm U make commision when SHE wins.. NICEE.. BTW.. What do u play when ure there?
1,@k80bug Ohhh I used to make those when I was operations there,0,Ohhh I used to make those when I was operations there
2,That is...if I can make it thru the west and southside of Montgomery without being stopped by the punkass cops.,0,That is...if I can make it thru the west and southside of Montgomery without being stopped by the punkass cops.
3,"Good stuff med @ericries Strax ï¿½ver en timme lï¿½ng, check it! http://bit.ly/15Vumv Tack @emileifrem !!",0,"Good stuff med Strax ï¿½ver en timme lï¿½ng, check it! Tack !!"
4,Tomorrow I begin a two day shoot for an online retailer. I'll be posing in various garments... I love it!,0,Tomorrow I begin a two day shoot for an online retailer. I'll be posing in various garments... I love it!
5,@hillkath Good morning.,0,Good morning.
6,"In Melbourne Australia, Thursday 7th May there is a Body Balance info night, tasting and free bottle! Direct message me email for details",0,"In Melbourne Australia, Thursday 7th May there is a Body Balance info night, tasting and free bottle! Direct message me email for details"
7,@therealswizzz http://twitpic.com/5eyah - XOXO,0,- XOXO
8,@SashaKane have a happy day,0,have a happy day
9,@VivaLaCaro That was coolio! &lt;3 I left reviews and fatty fuel.,0,That was coolio! &lt;3 I left reviews and fatty fuel.


In [11]:
#Step 4. Replace punctuations except for ! with whitespace
#Get punctuations
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [12]:
#Replace punctuation with whitespace
df['clean_text'] = df['clean_text'].apply(lambda x: re.sub("""['!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~']""",
                                                                  " ", x))
df.head(10)

C:\Users\Anuroop\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,text,label,clean_text
0,@vallywally LOL.. Damm U make commision when SHE wins.. NICEE.. BTW.. What do u play when ure there?,0,LOL Damm U make commision when SHE wins NICEE BTW What do u play when ure there
1,@k80bug Ohhh I used to make those when I was operations there,0,Ohhh I used to make those when I was operations there
2,That is...if I can make it thru the west and southside of Montgomery without being stopped by the punkass cops.,0,That is if I can make it thru the west and southside of Montgomery without being stopped by the punkass cops
3,"Good stuff med @ericries Strax ï¿½ver en timme lï¿½ng, check it! http://bit.ly/15Vumv Tack @emileifrem !!",0,Good stuff med Strax ï¿½ver en timme lï¿½ng check it Tack
4,Tomorrow I begin a two day shoot for an online retailer. I'll be posing in various garments... I love it!,0,Tomorrow I begin a two day shoot for an online retailer I ll be posing in various garments I love it
5,@hillkath Good morning.,0,Good morning
6,"In Melbourne Australia, Thursday 7th May there is a Body Balance info night, tasting and free bottle! Direct message me email for details",0,In Melbourne Australia Thursday 7th May there is a Body Balance info night tasting and free bottle Direct message me email for details
7,@therealswizzz http://twitpic.com/5eyah - XOXO,0,XOXO
8,@SashaKane have a happy day,0,have a happy day
9,@VivaLaCaro That was coolio! &lt;3 I left reviews and fatty fuel.,0,That was coolio lt 3 I left reviews and fatty fuel


In [13]:
#Remove numbers, code below will remove all non alphabets and convert to lowercase
df['clean_text'] = df['clean_text'].apply(lambda x: re.sub('[^A-Za-z]', ' ', x.lower()))
#^ = Negation
#Verify
df.head(10)

C:\Users\Anuroop\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,text,label,clean_text
0,@vallywally LOL.. Damm U make commision when SHE wins.. NICEE.. BTW.. What do u play when ure there?,0,lol damm u make commision when she wins nicee btw what do u play when ure there
1,@k80bug Ohhh I used to make those when I was operations there,0,ohhh i used to make those when i was operations there
2,That is...if I can make it thru the west and southside of Montgomery without being stopped by the punkass cops.,0,that is if i can make it thru the west and southside of montgomery without being stopped by the punkass cops
3,"Good stuff med @ericries Strax ï¿½ver en timme lï¿½ng, check it! http://bit.ly/15Vumv Tack @emileifrem !!",0,good stuff med strax ver en timme l ng check it tack
4,Tomorrow I begin a two day shoot for an online retailer. I'll be posing in various garments... I love it!,0,tomorrow i begin a two day shoot for an online retailer i ll be posing in various garments i love it
5,@hillkath Good morning.,0,good morning
6,"In Melbourne Australia, Thursday 7th May there is a Body Balance info night, tasting and free bottle! Direct message me email for details",0,in melbourne australia thursday th may there is a body balance info night tasting and free bottle direct message me email for details
7,@therealswizzz http://twitpic.com/5eyah - XOXO,0,xoxo
8,@SashaKane have a happy day,0,have a happy day
9,@VivaLaCaro That was coolio! &lt;3 I left reviews and fatty fuel.,0,that was coolio lt i left reviews and fatty fuel


In [26]:
###############Remove letters repeated more than twice in succession
#Create a function
def trim_repeated_letters(sentence):
    return(reduce(lambda x,y: x+y if x[-2:] != y*2 else x, sentence, ""))
    

#Apply function to sample_df
df['clean_text'] = df['clean_text'].apply(lambda x: trim_repeated_letters(x) )
df.head()

C:\Users\Anuroop\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,text,label,clean_text
0,@vallywally LOL.. Damm U make commision when SHE wins.. NICEE.. BTW.. What do u play when ure there?,0,lol damm u make commision when she wins nicee btw what do u play when ure there
1,@k80bug Ohhh I used to make those when I was operations there,0,ohh i used to make those when i was operations there
2,That is...if I can make it thru the west and southside of Montgomery without being stopped by the punkass cops.,0,that is if i can make it thru the west and southside of montgomery without being stopped by the punkass cops
3,"Good stuff med @ericries Strax ï¿½ver en timme lï¿½ng, check it! http://bit.ly/15Vumv Tack @emileifrem !!",0,good stuff med strax ver en timme l ng check it tack
4,Tomorrow I begin a two day shoot for an online retailer. I'll be posing in various garments... I love it!,0,tomorrow i begin a two day shoot for an online retailer i ll be posing in various garments i love it


All data cleaning steps used above will be compiled into one **df cleaning function** as it will have to be used multiple times

In [30]:
#Repeated letters func
def trim_repeated_letters(sentence):
    return(reduce(lambda x,y: x+y if x[-2:] != y*2 else x, sentence, ""))

#Compile allsteps above into one function for future use
def text_df_cleaner(df):
    '''
    Cleans 'text' column of a df and creates a new column 'clean_text' with clean text
    '''
    df['clean_text'] = df['text'].apply(lambda x: re.sub("@[\w]*","", x)) #User handles
    df['clean_text'] = df['clean_text'].apply(lambda x: re.sub("#[\w]*","", x)) #hash tags
    df['clean_text'] = df['clean_text'].apply(lambda x: re.sub("http\S+", "", x)) #urls
    df['clean_text'] = df['clean_text'].apply(lambda x: re.sub("""['!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~']""",
                                                                  " ", x)) #Punctuations
    df['clean_text'] = df['clean_text'].apply(lambda x: re.sub('[^A-Za-z]', ' ', x.lower())) #Numbers and lowercase
    #Repeated letters
    df['clean_text'] = df['clean_text'].apply(lambda x: trim_repeated_letters(x))
    return df
    

In [37]:
#Verify function
trial_df = master_df.tail(20)
text_df_cleaner(trial_df)
trial_df.head()
#Function is working properly

C:\Users\Anuroop\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Anuroop\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Anuroop\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

,text,label,clean_text
50478,"HATE safeway select green tea icecream! bought two cartons, what a waste of money. &gt;_&lt;",1,hate safeway select green tea icecream bought two cartons what a waste of money gt lt
50479,argghhhh why won't my jquery appear in safari bad safari !!!,1,argghh why won t my jquery appear in safari bad safari
50480,"I can't watch TV without a Tivo. And after all these years, the Time/Warner DVR STILL sucks. http://www.davehitt.com/march03/twdvr.html",1,i can t watch tv without a tivo and after all these years the time warner dvr still sucks
50481,I still love my Kindle2 but reading The New York Times on it does not feel natural. I miss the Bloomingdale ads.,1,i still love my kindle but reading the new york times on it does not feel natural i miss the bloomingdale ads
50482,"Although today's keynote rocked, for every great announcement, AT&amp;T shit on us just a little bit more.",1,although today s keynote rocked for every great announcement at amp t shit on us just a little bit more


In [37]:
a = 'http://twitpic.com/5eyah - XOXO'
re.sub("http\S+", "", a)

' - XOXO'

In [27]:
a = '@therealswizzz https://twitpic.com/5eyah - XOXO'
re.sub("http\S+", "", a)

'@therealswizzz  - XOXO'

In [6]:
#Get punctuations
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [7]:
#Replace punctuation with whitespace
sample_df['clean_text'] = sample_df['text'].apply(lambda x: re.sub("""['"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~']""",
                                                                  " ", x))

C:\Users\Anuroop\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [8]:
sample_df['clean_text']

0      vallywally LOL   Damm U make commision when SHE wins   NICEE    BTW   What do u play when ure there                                             
1      k80bug Ohhh I used to make those when I was operations there                                                                                    
2     That is   if I can make it thru the west and southside of Montgomery without being stopped by the punkass cops                                   
3     Good stuff med  ericries Strax ï¿½ver en timme lï¿½ng  check it! http   bit ly 15Vumv  Tack  emileifrem !!                                       
4     Tomorrow I begin a two day shoot for an online retailer  I ll be posing in various garments    I love it!                                        
5      hillkath Good morning                                                                                                                           
6     In Melbourne Australia  Thursday 7th May there is a Body Balance info night  tasti

In [ ]:
#Remove user handles 
master_df['clean_text'] = train['clean_tweet'].apply(lambda x: re.sub("@[\w]*", '', x))


In [32]:
master_df['url'] = master_df['text'].apply(lambda x: re.findall("http://[\w]*", x))

IndexError: list index out of range

In [30]:
master_df['url'].value_counts()

[]                                                                                                        49140
[http://twitpic]                                                                                          351  
[http://bit]                                                                                              269  
[http://tinyurl]                                                                                          138  
[http://www]                                                                                              93   
[http://plurk]                                                                                            79   
[http://blip]                                                                                             51   
[http://myloc]                                                                                            34   
[http://www, http://g, http://www, http://g, http://g, http://www, http://ecx, http://ecx, http://ecx]  

In [31]:
master_df[master_df['label'] == 0]['url'].value_counts()

[]                                                                                                        24277
[http://twitpic]                                                                                          268  
[http://bit]                                                                                              221  
[http://tinyurl]                                                                                          103  
[http://www]                                                                                              73   
[http://blip]                                                                                             47   
[http://plurk]                                                                                            45   
[http://yfrog]                                                                                            19   
[http://myloc]                                                                                          

In [25]:
sentiment_140_df['hashtags'] = sentiment_140_df['text'].apply(lambda x: re.findall("#[\w]*", x))

AttributeError: 'list' object has no attribute 'list'

In [15]:
sentiment_140_df[sentiment_140_df['label'] == 1]['hashtags'].value_counts()

[]                                                            774578
[#fb]                                                         995   
[#squarespace]                                                522   
[#]                                                           458   
[#iranelection]                                               213   
[#inaperfectworld]                                            170   
[#fail]                                                       164   
[#1]                                                          155   
[#iremember]                                                  134   
[#2]                                                          119   
[#BSB]                                                        111   
[#asot400]                                                    110   
[#followfriday]                                               103   
[#dontyouhate]                                                96    
[#haveyouever]                    

In [24]:
sentiment_140_df[sentiment_140_df['hashtags'] == '[#iPhone]']

,text,label,hashtags


In [9]:
#Potential test data
#master_df['hashtags'] = master_df['text'].apply(lambda x: re.findall("#[\w]*", x))

In [11]:
master_df['hashtags'].value_counts()

[]                                                     49340
[#]                                                    304  
[#, #]                                                 76   
[#followfriday]                                        34   
[#1]                                                   33   
[#2]                                                   23   
[#fb]                                                  23   
[#, #, #]                                              22   
[#FFFFFF, #FFFFFF]                                     22   
[#squarespace]                                         19   
[#FF]                                                  10   
[#, #, #, #]                                           9    
[#, #, #, #, #]                                        9    
[#BGT]                                                 7    
[#ff]                                                  7    
[#FollowFriday]                                        6    
[#bgt]                  

In [6]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [9]:
sample_df = master_df.tail(100).head(10)

In [27]:
sample_df['clean_text'] = sample_df['text'].apply(lambda x: re.findall("@[\w]*", x))

In [28]:
sample_df.head()

,text,label,clean_text
50398,Naive Bayes using EM for Text Classification. Really Frustrating...,1,[]
50399,"@KarrisFoxy If you're being harassed by calls about your car warranty, changing your number won't fix that. They call every number. #d-bags",1,[@KarrisFoxy]
50400,Just blocked United Blood Services using Google Voice. They call more than those Car Warranty guys.,1,[]
50401,#at&amp;t is complete fail.,1,[]
50402,@broskiii OH SNAP YOU WORK AT AT&amp;T DON'T YOU,1,[@broskiii]


In [30]:
master_df['clean_text'] = master_df['text'].apply(lambda x: re.findall("@[\w]*", x))

In [31]:
master_df['clean_text'].value_counts()

[]                              38254
[@]                             169  
[@mileycyrus]                   60   
[@tommcfly]                     60   
[@ddlovato]                     51   
[@JonathanRKnight]              27   
[@taylorswift13]                22   
[@selenagomez]                  21   
[@DavidArchie]                  20   
[@Jonasbrothers]                19   
[@mitchelmusso]                 18   
[@YoungQ]                       17   
[@, @]                          14   
[@DonnieWahlberg]               14   
[@jordanknight]                 13   
[@peterfacinelli]               13   
[@jonasbrothers]                10   
[@hotmail]                      10   
[@lilyroseallen]                10   
[@souljaboytellem]              10   
[@shaundiviney]                 9    
[@MariahCarey]                  8    
[@dannywood]                    8    
[@dougiemcfly]                  8    
[@trent_reznor]                 8    
[@aplusk]                       8    
[@officialTi

In [4]:
sentiment_140_df['tags'] = sentiment_140_df['text'].apply(lambda x: re.findall("@[\w]*", x))

In [ ]:
sentiment_140_df['tags'].value_counts()

In [ ]:
sentiment_140_df[sentiment_140_df['label'] == 0]['tags'].value_counts()

In [39]:
s="haaaappppyy"

In [45]:
reduce(lambda x,y: x+y,s)

'haaaappppyy'

In [24]:
reduce(lambda x,y: x+y if x[-2:]!=y*2 else x, s, "")


'haappyy'

In [ ]:
reduce(lambda x,y: x+y if x[-2:]!=y*2 else x, s, "")
'haappyy'